In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import joblib

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D, LSTM
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torcheval.metrics import MultilabelAccuracy
import torch
from prettytable import PrettyTable

from matplotlib import pyplot as plt

In [ ]:
train_df = joblib.load('data/train_df.joblib')
test_df = joblib.load('data/test_df.joblib')

In [ ]:
category_idx_to_text = {0: 'ambience', 1: 'food', 2: 'other', 3: 'price', 4: 'service'}
polarity_idx_to_text = {0: 'negative', 1: 'neutral', 2: 'positive'}
joint_idx_to_text = {0: 'ambience#negative', 1: 'ambience#neutral', 2: 'ambience#positive', 3: 'food#negative', 4: 'food#neutral', 5: 'food#positive', 6: 'other#negative', 7: 'other#neutral', 8: 'other#positive', 9: 'price#negative', 10: 'price#neutral', 11: 'price#positive', 12: 'service#negative', 13: 'service#neutral', 14: 'service#positive'}

In [ ]:
(train_df['text'].apply(len) > 180).sum()

6

In [ ]:
maxlen = 180   # only 6 reviews longer than 180 char
max_words = 5000
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(train_df["text"])

def get_features(text_series):
    """
    transforms text data to feature_vectors that can be used in the ml model.
    tokenizer must be available.
    """
    sequences = tokenizer.texts_to_sequences(text_series)
    return pad_sequences(sequences, maxlen=maxlen)


def prediction_to_label(prediction):
    tag_prob = [(labels[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=True))

In [ ]:
class LSTMmodel:
    def __init__(self, num_classes):
        filter_length = 300

        model = Sequential()
        model.add(Embedding(max_words, 20, input_length=maxlen))
        model.add(Dropout(0.1))
        model.add(LSTM(filter_length, return_sequences=True))
        model.add(GlobalMaxPool1D())
        model.add(Dense(num_classes))
        model.add(Activation('sigmoid'))

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
        model.summary()

        self.model = model

In [ ]:
def model_metrics(test_labels, predictions):
    accuracy = accuracy_score(test_labels, predictions)

    exact_accuracy = MultilabelAccuracy(criteria='exact_match')
    exact_accuracy.update(target=torch.Tensor(test_labels), input=torch.Tensor(predictions))

    overlap_accuracy = MultilabelAccuracy(criteria='overlap')
    overlap_accuracy.update(target=torch.Tensor(test_labels), input=torch.Tensor(predictions))

    macro_f1 = f1_score(test_labels, predictions, average='macro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')

    metrics = {}
    metrics = {
        'accuracy' : accuracy,
        'exact_match_accuracy': exact_accuracy.compute().detach().item(),
        'overlap_accuracy': overlap_accuracy.compute().detach().item(),
        'macro_f1': macro_f1,
        'micro_f1': micro_f1
    }

    return metrics

def pretty_table(dict):
    table = PrettyTable()
    table.field_names = ['metric', 'value']
    for k,v in dict.items():
        table.add_row([k, v])
    print(table)

In [ ]:
def plot_history(history, path):
    # Create a figure and a 1x2 grid of subplots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

    # Plot the first subplot on the left
    axes[0].plot(history.history['categorical_accuracy'])
    axes[0].plot(history.history['val_categorical_accuracy'])
    axes[0].set_title('model categorical accuracy')
    axes[0].set_ylabel('accuracy')
    axes[0].set_xlabel('epoch')
    axes[0].legend(['train', 'val'], loc='upper left')

        # Plot the first subplot on the left
    axes[1].plot(history.history['loss'])
    axes[1].plot(history.history['val_loss'])
    axes[1].set_title('model loss')
    axes[1].set_ylabel('loss')
    axes[1].set_xlabel('epoch')
    axes[1].legend(['train', 'val'], loc='upper left')

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.savefig(path)
    plt.show()

## Category classification

In [ ]:
category_labels = category_idx_to_text.values()

x_train_category = get_features(train_df["text"])
x_test_category = get_features(test_df["text"])

y_train_category = np.array(train_df["category_labels"].tolist())
y_test_category = np.array(test_df["category_labels"].tolist())

In [ ]:
num_classes  = 5

callbacks = [
            ReduceLROnPlateau(),
            EarlyStopping(patience=4),
            ModelCheckpoint(filepath='category_model.model', save_best_only=True)
        ]

category_model = LSTMmodel(num_classes)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 180, 20)           100000    
                                                                 
 dropout (Dropout)           (None, 180, 20)           0         
                                                                 
 lstm (LSTM)                 (None, 180, 300)          385200    
                                                                 
 global_max_pooling1d (Glob  (None, 300)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 5)                 1505      
                                                                 
 activation (Activation)     (None, 5)                 0         
                                                        

In [ ]:
history_category = category_model.model.fit(x_train_category, y_train_category,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/20
81/81 [==============================] - 22s 207ms/step - loss: 0.5287 - categorical_accuracy: 0.3545 - val_loss: 0.5096 - val_categorical_accuracy: 0.3636 - lr: 0.0010
Epoch 2/20
81/81 [==============================] - 10s 118ms/step - loss: 0.5131 - categorical_accuracy: 0.3701 - val_loss: 0.5059 - val_categorical_accuracy: 0.3636 - lr: 0.0010
Epoch 3/20
81/81 [==============================] - 6s 75ms/step - loss: 0.5018 - categorical_accuracy: 0.4897 - val_loss: 0.4857 - val_categorical_accuracy: 0.5524 - lr: 0.0010
Epoch 4/20
81/81 [==============================] - 7s 87ms/step - loss: 0.4504 - categorical_accuracy: 0.6404 - val_loss: 0.4220 - val_categorical_accuracy: 0.6259 - lr: 0.0010
Epoch 5/20
81/81 [==============================] - 6s 67ms/step - loss: 0.3544 - categorical_accuracy: 0.6961 - val_loss: 0.3816 - val_categorical_accuracy: 0.6329 - lr: 0.0010
Epoch 6/20
81/81 [==============================] - 5s 67ms/step - loss: 0.2878 - categorical_accuracy: 0.

In [ ]:
lstm_model_category = keras.models.load_model('category_model.model')

In [ ]:
lstm_model_category.evaluate(x_test_category, y_test_category)

24/24 [==============================] - 1s 11ms/step - loss: 0.3150 - categorical_accuracy: 0.6862


[0.31499937176704407, 0.6862483024597168]

In [ ]:
predict_category = lstm_model_category.predict(x_test_category)
threshold = 0.5

# Convert to binary format
predict_category = (predict_category > threshold).astype(int)

24/24 [==============================] - 1s 7ms/step


In [ ]:
metrics_category = model_metrics(predict_category, y_test_category)

In [ ]:
pretty_table(metrics_category)

+----------------------+--------------------+
|        metric        |       value        |
+----------------------+--------------------+
|       accuracy       | 0.5834445927903872 |
| exact_match_accuracy | 0.5834445953369141 |
|   overlap_accuracy   | 0.7636849284172058 |
|       macro_f1       | 0.5011922015182886 |
|       micro_f1       | 0.7412333736396615 |
+----------------------+--------------------+


In [ ]:
plot_history(history_category, '/lstm_category_plot.png')

## Sentiment polarity

In [ ]:
polarity_labels = polarity_idx_to_text.values()

x_train_polarity = get_features(train_df["text"])
x_test_polarity = get_features(test_df["text"])

y_train_polarity = np.array(train_df["polarity_labels"].tolist())
y_test_polarity = np.array(test_df["polarity_labels"].tolist())

In [ ]:
polarity_labels

dict_values(['negative', 'neutral', 'positive'])

In [ ]:
num_classes  = len(polarity_labels)

callbacks = [
            ReduceLROnPlateau(),
            EarlyStopping(patience=4),
            ModelCheckpoint(filepath='polarity_model.model', save_best_only=True)
        ]

polarity_model = LSTMmodel(num_classes)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 180, 20)           100000    
                                                                 
 dropout_1 (Dropout)         (None, 180, 20)           0         
                                                                 
 lstm_1 (LSTM)               (None, 180, 300)          385200    
                                                                 
 global_max_pooling1d_1 (Gl  (None, 300)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_1 (Dense)             (None, 3)                 903       
                                                                 
 activation_1 (Activation)   (None, 3)                 0         
                                                      

In [ ]:
history_polarity = polarity_model.model.fit(x_train_polarity, y_train_polarity,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/20
81/81 [==============================] - 14s 141ms/step - loss: 0.5710 - categorical_accuracy: 0.5820 - val_loss: 0.5895 - val_categorical_accuracy: 0.5629 - lr: 0.0010
Epoch 2/20
81/81 [==============================] - 6s 77ms/step - loss: 0.5582 - categorical_accuracy: 0.5863 - val_loss: 0.5684 - val_categorical_accuracy: 0.5629 - lr: 0.0010
Epoch 3/20
81/81 [==============================] - 7s 85ms/step - loss: 0.5150 - categorical_accuracy: 0.5952 - val_loss: 0.5506 - val_categorical_accuracy: 0.4965 - lr: 0.0010
Epoch 4/20
81/81 [==============================] - 5s 65ms/step - loss: 0.4066 - categorical_accuracy: 0.7098 - val_loss: 0.4950 - val_categorical_accuracy: 0.6399 - lr: 0.0010
Epoch 5/20
81/81 [==============================] - 2s 18ms/step - loss: 0.3268 - categorical_accuracy: 0.7663 - val_loss: 0.5303 - val_categorical_accuracy: 0.6469 - lr: 0.0010
Epoch 6/20
81/81 [==============================] - 2s 27ms/step - loss: 0.2707 - categorical_accuracy: 0.81

In [ ]:
lstm_model_polarity = keras.models.load_model('polarity_model.model')

In [ ]:
lstm_model_polarity.evaluate(x_test_polarity, y_test_polarity)

24/24 [==============================] - 1s 10ms/step - loss: 0.3941 - categorical_accuracy: 0.7356


[0.3941097855567932, 0.7356475591659546]

In [ ]:
predict_polarity = lstm_model_polarity.predict(x_test_polarity)
threshold = 0.5

# Convert to binary format
predict_polarity = (predict_polarity > threshold).astype(int)

24/24 [==============================] - 1s 7ms/step


In [ ]:
metrics_polarity = model_metrics(predict_polarity, y_test_polarity)

In [ ]:
pretty_table(metrics_polarity)

+----------------------+--------------------+
|        metric        |       value        |
+----------------------+--------------------+
|       accuracy       | 0.6261682242990654 |
| exact_match_accuracy | 0.6261682510375977 |
|   overlap_accuracy   | 0.644859790802002  |
|       macro_f1       | 0.4078508319753456 |
|       micro_f1       | 0.7129151291512915 |
+----------------------+--------------------+


In [ ]:
plot_history(history_polarity, 'lstm_polarity_plot.png')

## Joint classification

In [ ]:
joint_labels = joint_idx_to_text.values()

x_train_joint = get_features(train_df['text'])
x_test_joint = get_features(test_df['text'])

y_train_joint = np.array(train_df['joint_labels'].tolist())
y_test_joint = np.array(test_df['joint_labels'].tolist())

In [ ]:
joint_labels

dict_values(['ambience#negative', 'ambience#neutral', 'ambience#positive', 'food#negative', 'food#neutral', 'food#positive', 'other#negative', 'other#neutral', 'other#positive', 'price#negative', 'price#neutral', 'price#positive', 'service#negative', 'service#neutral', 'service#positive'])

In [ ]:
num_classes  = len(joint_labels)

callbacks = [
            ReduceLROnPlateau(),
            EarlyStopping(patience=4),
            ModelCheckpoint(filepath='joint_model.model', save_best_only=True)
        ]

joint_model = LSTMmodel(num_classes)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 180, 20)           100000    
                                                                 
 dropout_2 (Dropout)         (None, 180, 20)           0         
                                                                 
 lstm_2 (LSTM)               (None, 180, 300)          385200    
                                                                 
 global_max_pooling1d_2 (Gl  (None, 300)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 15)                4515      
                                                                 
 activation_2 (Activation)   (None, 15)                0         
                                                      

In [ ]:
history_joint = joint_model.model.fit(x_train_joint, y_train_joint,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/20
81/81 [==============================] - 15s 154ms/step - loss: 0.2871 - categorical_accuracy: 0.2462 - val_loss: 0.2476 - val_categorical_accuracy: 0.2727 - lr: 0.0010
Epoch 2/20
81/81 [==============================] - 4s 45ms/step - loss: 0.2484 - categorical_accuracy: 0.2641 - val_loss: 0.2487 - val_categorical_accuracy: 0.2727 - lr: 0.0010
Epoch 3/20
81/81 [==============================] - 2s 28ms/step - loss: 0.2477 - categorical_accuracy: 0.2676 - val_loss: 0.2482 - val_categorical_accuracy: 0.2727 - lr: 0.0010
Epoch 4/20
81/81 [==============================] - 6s 75ms/step - loss: 0.2475 - categorical_accuracy: 0.2707 - val_loss: 0.2463 - val_categorical_accuracy: 0.2727 - lr: 0.0010
Epoch 5/20
81/81 [==============================] - 5s 61ms/step - loss: 0.2435 - categorical_accuracy: 0.2922 - val_loss: 0.2405 - val_categorical_accuracy: 0.2762 - lr: 0.0010
Epoch 6/20
81/81 [==============================] - 6s 70ms/step - loss: 0.2355 - categorical_accuracy: 0.33

In [ ]:
lstm_model_joint = keras.models.load_model('joint_model.model')

In [ ]:
lstm_model_joint.evaluate(x_test_joint, y_test_joint)

24/24 [==============================] - 1s 11ms/step - loss: 0.1910 - categorical_accuracy: 0.4980


[0.1909979283809662, 0.49799734354019165]

In [ ]:
predict_joint = lstm_model_joint.predict(x_test_joint)
threshold = 0.5

# Convert to binary format
predict_joint = (predict_joint > threshold).astype(int)

24/24 [==============================] - 1s 9ms/step


In [ ]:
metrics_joint = model_metrics(predict_joint, y_test_joint)

In [ ]:
pretty_table(metrics_joint)

+----------------------+---------------------+
|        metric        |        value        |
+----------------------+---------------------+
|       accuracy       |  0.3164218958611482 |
| exact_match_accuracy | 0.31642189621925354 |
|   overlap_accuracy   |  0.4419225752353668 |
|       macro_f1       | 0.17767885291063154 |
|       micro_f1       |  0.504225352112676  |
+----------------------+---------------------+


In [ ]:
plot_history(history_joint, 'lstm_joint_plot.png')